In [1]:
# -*- coding: utf-8 -*-
import pymongo
import urllib

def get_mongo_db_client():
    username_str = 'breadt'
    password_str = 'Breadt@2019'

    username = urllib.parse.quote_plus(username_str)
    password = urllib.parse.quote_plus(password_str)

    client = pymongo.MongoClient('mongodb://%s:%s@10.12.86.109:27017/' % (username, password))
    return client

In [95]:
# -*- coding: utf-8 -*-
import pymongo
import pymysql
from lxml import etree
import re
import pandas as pd
import numpy as np

In [118]:
def get_data_from_db(date):
    client = get_mongo_db_client()
    db = client['ke']

    data = []
    for r in db['kelist'].find({'date': date}):

        content = re.sub(r'<script[^>]*?>(?:.|\n)*?<\/script>', '', r['content'])
        content = re.sub(r'<meta.+>', '', content)
        content = re.sub(r'<link.+>', '', content)
        content = re.sub(r'<!--.+-->', '', content)
        content = re.sub(r'\n', '', content)
        content = re.sub(r'<br/>', '', content)
        content = re.sub(r'<span class="houseIcon"></span>', '', content)

        pattern = re.compile(r'<li class="clear">.*?</li>')   # 查找数字
        result = pattern.findall(content)

        for item in result:
            html = etree.HTML(item)
            
            house_info = html.xpath('//div[@class="houseInfo"]/text()')[0].replace(' ', '')
            house_info_arr = house_info.split('|')
            
            if len(house_info_arr) == 5:
                year = house_info_arr[1].replace('年建', '')
            elif len(house_info_arr) == 4:
                year = house_info_arr[1].replace('年建', '')
            else:
                year = 0
            
            if len(house_info_arr) == 5:
                size = house_info_arr[3].replace('平米', '')
            elif len(house_info_arr) == 3:
                size = house_info_arr[1].replace('平米', '')
            else:
                size = house_info_arr[-1].replace('平米', '')
                
            if len(house_info_arr) == 5:
                structure = house_info_arr[2]
            elif len(house_info_arr) == 4:
                structure = house_info_arr[2]
            else:
                structure = house_info_arr[0].split(')')[1]
                
            if len(house_info_arr) == 5:
                op = house_info_arr[4]
            elif len(house_info_arr) == 3:
                op = house_info_arr[2]
            else:
                op = 0
            
            if len(house_info_arr) == 5:
                level = house_info_arr[0]
            elif len(house_info_arr) == 3:
                level = house_info_arr[0].split(')')[0]
            else:
                level = house_info_arr[0]
                
            tax_info = html.xpath('//span[@class="taxfree"]/text()')
            
            data.append({
                'href': html.xpath('//a[contains(@class,"maidian-detail")]/@href')[0],
                'title':html.xpath('//a[contains(@class,"maidian-detail")]/@title')[0],
                'info': house_info,
                'level': level,
                'year': year,
                'structure': structure,
                'size': size,
                'op': op,
                'price': html.xpath('//div[@class="totalPrice"]/span/text()')[0],
                'tax': tax_info[0] if len(tax_info) > 0 else 0,
                'date': r['date'],
                'address': html.xpath('//div[@class="positionInfo"]//a/text()')[0],
            })
    return data

In [125]:
data = get_data_from_db('2019-07-03')
df = pd.DataFrame(data)

In [126]:
df.columns.values

array(['address', 'date', 'href', 'info', 'level', 'op', 'price', 'size',
       'structure', 'tax', 'title', 'year'], dtype=object)

In [127]:
t = df[['address', 'level', 'op', 'price', 'size','structure', 'tax', 'title', 'year']]
t['price'] = t['price'].astype(np.float32)
t['size'] = t['size'].astype(np.float32)
t['avg_price'] = t['price'] / t['size']
t[ (t['price']>400) & (t['price']<700)].head(20)

/home/leewind/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/leewind/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/leewind/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,address,level,op,price,size,structure,tax,title,year,avg_price
6,乐业公寓,低楼层(共20层),南,520.0,83.290001,2室1厅,0,乐业公寓+电梯两房+近地铁8号线+诚意出售+看房方便,1999,6.243247
10,中心铂庭,中楼层(共27层),南,450.0,97.000000,2室1厅,0,全南方正户型 看苏州河 近地铁大悦城 二手买进税少,2010,4.639175
13,世纪花苑,低楼层(共6层),南,468.0,89.680000,2室1厅,满五年,世纪花苑经典2室户+房东诚意出售+到价签字+看房方便,1999,5.218555
16,世纪花苑,中楼层(共7层),南,586.0,100.019997,2室2厅,0,满五唯一，小高层+电梯房，楼层好位置佳，诚意出售,1999,5.858829
20,星保花苑,低楼层(共13层),南,438.0,93.650002,3室1厅,0,新静安区电梯房精装修房东诚心出售,1993,4.676989
24,绿苑(闸北),低楼层(共6层),南,520.0,100.839996,3室2厅,满五年,此房是多层2楼，全明户型，房型方正，看房方便,1997,5.156684
33,久乐苑,中楼层(共7层),南,565.0,90.839996,2室1厅,0,久乐苑 2室1厅 南,1998,6.219727
42,中兴财富国际公寓,中楼层(共34层),南西,605.0,79.739998,2室2厅,0,内环内2005年住宅 全明户型 产权清晰 看房方便哦,2005,7.587159
57,大宁龙盛雅苑,中楼层(共21层),西北,518.0,62.029999,2室1厅,0,电梯房 房龄新 价格合理 地铁一号线延长路站,2012,8.350799
61,绿苑(闸北),高楼层(共6层),南,520.0,101.720001,3室2厅,0,灵石公园g+总价低大三房+业主诚意出售,1997,5.112072


In [147]:
t[t['address'].str.find('长亭')>0]

,address,level,op,price,size,structure,tax,title,year,avg_price


In [139]:
t[t['address'] == '永盛苑']

,address,level,op,price,size,structure,tax,title,year,avg_price
243,永盛苑,中楼层(共32层),南北,805.0,122.019997,2室2厅,满五年,苏州河畔，3轨交，满五唯一，税费少，婚房装修！,2003,6.597280
501,永盛苑,中楼层(共32层),南,800.0,121.830002,3室1厅,满五年,火车站，大悦城，不夜城公园，汉中路地铁，永盛苑,2003,6.566527
975,永盛苑,中楼层(共32层),东南,838.0,123.309998,2室2厅,满五年,苏河湾板块，大悦城商圈 近地铁 不夜城公园,2003,6.795880
1314,永盛苑,中楼层(共32层),南北,805.0,121.830002,2室2厅,0,内环苏河湾板块 次新社区3轨交汇,2003,6.607568
1962,永盛苑,低楼层(共32层),南,1030.0,154.529999,3室2厅,0,税费各付！+全南大三房+有产权车位+04年商品房,2003,6.665372


In [141]:
t[t['address'].str.find('苏河')>0].sort_values(['avg_price'])

,address,level,op,price,size,structure,tax,title,year,avg_price
2325,华侨城苏河湾(商业类),中楼层(共22层),南,1000.0,161.130005,2室2厅,0,华侨城苏河湾(商业类) 2室2厅 南,2012,6.206169
455,华侨城苏河湾(商业类),高楼层(共22层),南,1050.0,160.500000,2室2厅,0,高区景观房 正看苏州河 央企品质 看房随时 诚意出售,2012,6.542056
2340,华侨城苏河湾(商业类),中楼层(共22层),南,1100.0,161.130005,2室1厅,满五年,华侨城苏河湾(商业类) 2室1厅 南,2012,6.826786
2283,华侨城苏河湾(商业类),低楼层(共23层,南,1250.0,178.559998,3室2厅,0,坐拥苏州河，俯瞰黄浦江，沿苏州河品质小区,0,7.000448
2326,华侨城苏河湾(商业类),中楼层(共22层),南,1090.0,151.800003,2室2厅,满五年,位于上海母亲河苏州河，近地铁，烫金地段，交通便利,2012,7.180501
2154,华侨城苏河湾(商业类),低楼层(共23层,南,1200.0,164.690002,3室2厅,0,品质二期，看苏州河，屋况保养好，干净清爽,0,7.286417
2335,华侨城苏河湾(商业类),高楼层(共22层),南北,1190.0,162.759995,2室2厅,0,华侨城苏河湾(商业类) 2室2厅 南 北,2012,7.311379
2341,华侨城苏河湾(商业类),低楼层(共23层,南,1180.0,159.649994,2室2厅,0,华侨城苏河湾(商业类) 2室2厅 南,0,7.391169
2338,华侨城苏河湾(商业类),低楼层(共22层,南,1200.0,162.000000,2室2厅,0,华侨城苏河湾(商业类) 2室2厅 南,0,7.407407
2331,华侨城苏河湾(商业类),低楼层(共23层),南,1200.0,162.000000,3室2厅,0,华侨城苏河湾(商业类) 3室2厅 南,2012,7.407407


一中心的学区范围

![](../img/一中心.jpeg)